In [11]:
import os

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import tensorflow
from sklearn.metrics import auc, roc_curve
from tensorflow.keras.metrics import *
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.data.Dataset as dt

In [12]:
required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

!echo "PYTHONPATH: $PYTHONPATH"

PYTHONPATH: :/Users/carlos/WORKSPACE/MegaClassifier/data/external/ai4eutils:/Users/carlos/WORKSPACE/MegaClassifier/data/external/CameraTraps:/Users/carlos/WORKSPACE/MegaClassifier/data/external/yolov5


In [13]:
models = [
    ("MobileNetV2", "v.1.0"),
    ("MobileNetV2", "v.2.0"),
]

In [14]:
IMAGES_DATASET_PATH = os.path.abspath("./data/processed/original")
SAMPLES_CSV = os.path.abspath("./data/processed/original/28560Images_detection.csv")
dataframe = dt.load_from_csv(SAMPLES_CSV)
test_dataframe = dataframe[dataframe["subset"] == "test"]
test_dataframe["file_name"] = test_dataframe["file_name"].apply(
    lambda x: os.path.join(IMAGES_DATASET_PATH, x)
)
test_dataframe["binary_label"] = test_dataframe["binary_label"].astype(str)
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
normal_datagen = ImageDataGenerator(
    preprocessing_function=tensorflow.keras.applications.mobilenet_v2.preprocess_input
)
test_images = normal_datagen.flow_from_dataframe(
    dataframe=test_dataframe,
    x_col="file_name",
    y_col="binary_label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False,
)

The file /Users/carlos/WORKSPACE/MegaClassifier/data/processed/original/28560Images_detection.csv has been successfully opened.
Found 4286 validated image filenames belonging to 2 classes.


/var/folders/3k/sxxpr3712v34r3p9x8yl60sm0000gn/T/ipykernel_4692/3550688241.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/3k/sxxpr3712v34r3p9x8yl60sm0000gn/T/ipykernel_4692/3550688241.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
def plot_roc_curve(models, test_images):
    y_true = test_images.labels
    fig = go.Figure()

    for model_name, version in models:
        complete_name = f"{model_name}_{version}"
        model = load_model(f"./models/{model_name}/{complete_name}.h5")
        y_pred = model.predict(test_images)

        fpr, tpr, _ = roc_curve(y_true, y_pred)
        roc_auc = auc(fpr, tpr)
        version_short = version[:-2]

        fig.add_trace(
            go.Scatter(
                x=fpr,
                y=tpr,
                mode="lines",
                name=f"{model_name} {version_short} (AUC = {roc_auc:.4f})",
            )
        )

    # Línea de referencia (Random Guess)
    fig.add_trace(
        go.Scatter(
            x=[0, 1],
            y=[0, 1],
            mode="lines",
            name="Random Classifier",
            line=dict(dash="dash"),
        )
    )

    # Configurar diseño
    fig.update_layout(
        title="ROC Curve - Compare Models",
        xaxis_title="False Positive Rate (FPR)",
        yaxis_title="True Positive Rate (TPR)",
        legend_title="Curves",
        template="seaborn",
        width=700,
        height=500,
        xaxis=dict(
            tickmode="linear",
            tick0=0,
            dtick=0.1,
        ),
    )

    # Mostrar la gráfica
    fig.show()
    # fig.write_image(f"./reports/figures/roc_curve/compare_test.png")

In [16]:
def plot_test_metrics(models):
    fig = go.Figure()

    for model_name, version in models:
        test = dt.load_from_csv(
            os.path.abspath(f"./logs/{model_name}/{version}/test_history.csv"), sep=","
        )
        test = test.iloc[-1]

        colums = ["Accuracy", "Loss", "Precision", "Recall", "Specificity"]

        accuracy = (test["test_accuracy"] * 100).round(2)
        loss = (test["test_loss"] * 100).round(2)
        precision = (test["test_precision"] * 100).round(2)
        recall = (test["test_recall"] * 100).round(2)
        specificity = (
            test["test_tn"] / (test["test_tn"] + test["test_fp"]) * 100
        ).round(2)

        data = [accuracy, loss, precision, recall, specificity]

        fig.add_trace(
            go.Bar(
                name=f"{model_name} {version}",
                x=colums,
                y=data,
                text=data,
                textposition="auto",
            )
        )

    fig.update_layout(
        title="Test Metrics comparison",
        xaxis_title="Metrics",
        yaxis_title="Values (%)",
        template="seaborn",
        width=900,
        height=500,
        xaxis=dict(
            tickmode="linear",
            tick0=0,
            dtick=0.1,
        ),
    )

    fig.show()

In [17]:
def plot_test_metrics_2(models):
    fig = go.Figure()

    colums = ["Accuracy", "Precision", "NPV", "Recall", "Specificity"]

    model_metrics = pd.read_csv("reports/metrics/model_metrics.csv")
    selected = model_metrics.iloc[1]
    data = [
        (selected["accuracy"] * 100).round(2),
        (selected["precision"] * 100).round(2),
        (selected["NPV"] * 100).round(2),
        (selected["recall"] * 100).round(2),
        (selected["specificity"] * 100).round(2),
    ]

    fig.add_trace(
        go.Bar(
            name="MegaDetector Threshold=0.003",
            x=colums,
            y=data,
            text=data,
            textposition="auto",
        )
    )

    for model_name, version in models:
        test = dt.load_from_csv(
            os.path.abspath(f"./logs/{model_name}/{version}/test_history.csv"), sep=","
        )
        TP = test["test_tp"].iloc[0]
        TN = test["test_tn"].iloc[0]
        FP = test["test_fp"].iloc[0]
        FN = test["test_fn"].iloc[0]

        accuracy = (TP + TN) / (TP + TN + FP + FN)
        precision = TP / (TP + FP)
        npv = TN / (TN + FN)
        recall = TP / (TP + FN)
        specificity = TN / (TN + FP)

        data = [accuracy, precision, npv, recall, specificity]
        data = [round(d * 100, 2) for d in data]

        fig.add_trace(
            go.Bar(
                name=f"{model_name} {version}",
                x=colums,
                y=data,
                text=data,
                textposition="auto",
            )
        )

    default = model_metrics.iloc[-1]
    data = [
        (default["accuracy"] * 100).round(2),
        (default["precision"] * 100).round(2),
        (default["NPV"] * 100).round(2),
        (default["recall"] * 100).round(2),
        (default["specificity"] * 100).round(2),
    ]

    fig.add_trace(
        go.Bar(
            name="MegaDetector Threshold=0.1",
            x=colums,
            y=data,
            text=data,
            textposition="auto",
        )
    )

    fig.update_layout(
        title="Test Metrics comparison with MegaDetector",
        xaxis_title="Metrics",
        yaxis_title="Values (%)",
        template="seaborn",
        width=1100,
        height=500,
        xaxis=dict(
            tickmode="linear",
            tick0=0,
            dtick=0.1,
        ),
    )

    fig.show()

In [18]:
plot_roc_curve(models, test_images)

2024-12-29 10:37:19.794852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


134/134 [==============================] - 21s 150ms/step


2024-12-29 10:37:41.006699: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


134/134 [==============================] - 20s 146ms/step


In [19]:
plot_test_metrics(models)

The file /Users/carlos/WORKSPACE/MegaClassifier/logs/MobileNetV2/v.1.0/test_history.csv has been successfully opened.
The file /Users/carlos/WORKSPACE/MegaClassifier/logs/MobileNetV2/v.2.0/test_history.csv has been successfully opened.


In [20]:
plot_test_metrics_2(models)

The file /Users/carlos/WORKSPACE/MegaClassifier/logs/MobileNetV2/v.1.0/test_history.csv has been successfully opened.
The file /Users/carlos/WORKSPACE/MegaClassifier/logs/MobileNetV2/v.2.0/test_history.csv has been successfully opened.
